# **Google Colab Notebook for CxC 2025 SAP Challenge**

# Step 1: Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
import shap
import plotly.express as px

# Step 2: Load Data (Update File Path)



In [ ]:
import os

# Define file path (Colab upload path)
data_file = '/content/SAP.xlsx'

# Ensure the file exists
if os.path.exists(data_file):
    xls = pd.ExcelFile(data_file)
    df = pd.read_excel(xls, sheet_name='AFE')
else:
    raise FileNotFoundError("File 'SAP.xlsx' not found in Colab uploads. Please upload it first.")

# Display basic information
df.head()

# Step 3: Select Key Indicators (Education, Health, Living Standards)

In [ ]:
mpi_indicators = [
    "Literacy rate, youth female (% of females ages 15-24)",
    "Literacy rate, youth (ages 15-24), gender parity index (GPI)",
    "Adjusted savings: education expenditure (current US$)",
    "Access to clean fuels and technologies for cooking (% of population)",
    "Access to electricity (% of population)",
    "Adequacy of social protection and labor programs (% of total welfare of beneficiary households)",
    "Multilateral debt service (% of public and publicly guaranteed debt service)"
]

mpi_df = df[df["Indicator Name"].isin(mpi_indicators)].copy()
mpi_df = mpi_df.drop(columns=["Topic", "short description", "long description", "Indicator Code", "Unit of measure"])
mpi_df = mpi_df.melt(id_vars=["Country Name", "Country Code", "Indicator Name"], var_name="Year", value_name="Value")
mpi_df["Year"] = mpi_df["Year"].astype(int)
mpi_df = mpi_df.sort_values(by=["Country Name", "Year"])


# Step 4: Handle Missing Values


In [ ]:
imputer = SimpleImputer(strategy='median')
mpi_df['Value'] = imputer.fit_transform(mpi_df[['Value']])

# Step 5: Normalize the Data

In [ ]:
scaler = MinMaxScaler()
mpi_df['Normalized Value'] = scaler.fit_transform(mpi_df[['Value']])

# Step 6: Perform Feature Selection using PCA

In [ ]:
pivot_df = mpi_df.pivot(index=["Country Name", "Year"], columns="Indicator Name", values="Normalized Value")
pca = PCA(n_components=3)
principal_components = pca.fit_transform(pivot_df.dropna())
mpi_df_pca = pd.DataFrame(principal_components, index=pivot_df.dropna().index, columns=["PC1", "PC2", "PC3"])

# Step 7: Build the MPI Index using Weighted Sum of PCA Components


In [ ]:
mpi_df_pca["MPI Index"] = mpi_df_pca["PC1"] * 0.5 + mpi_df_pca["PC2"] * 0.3 + mpi_df_pca["PC3"] * 0.2

# Step 8: Perform Regression Analysis to Determine Impact


In [ ]:
regressor = LinearRegression()
y = mpi_df_pca["MPI Index"]
X = pivot_df.dropna()
regressor.fit(X, y)

# Step 9: Use Clustering to Find Patterns

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
mpi_df_pca["Cluster"] = kmeans.fit_predict(mpi_df_pca)

# Step 10: Generate Insights and Policy Recommendations

In [ ]:
feature_importance_model = RandomForestRegressor()
feature_importance_model.fit(X, y)
shap_values = shap.Explainer(feature_importance_model)(X)
shap.summary_plot(shap_values, X)

# Step 11: Visualize Results

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(mpi_df_pca.pivot_table(index="Country Name", columns="Year", values="MPI Index"), cmap="coolwarm")
plt.title("Heatmap of MPI Index by Country and Year")
plt.show()

fig = px.scatter(mpi_df_pca, x="PC1", y="PC2", color="Cluster", hover_name=mpi_df_pca.index.get_level_values("Country Name"))
fig.show()